In [2]:
import pandas as pd
import numpy
import codecs
import csv
import json
import pprint
import re

filename = "math sample for API testing.csv"

fields ={'Internal_ID': 'Internal_ID',
         'Strand': 'Strand',
         'Sub-Strand': 'Substrand',
         'Content Focus': 'Content_Focus',
         'Component': 'Component',
         'Number': 'Number',
         'Digits': 'Digits',
         'Shape': 'Shape',
         'Units': 'Units',
         'Steps': 'Steps',
         'Context': 'Context',
         'SIG Stem': 'SIG_Stem',
         'SIG Options': 'SIG_Options',
         'Additional Modifiers': 'Additional_Modifiers'}

In [3]:
def process_file(filename, fields, tagging_df):
    process_fields = fields.keys()
    data = []
    bad_data = []
#    data_keys = ['Internal ID', 'Strand','Substrand', 'Content_focus', 'Component', 'Number', 'Digits', 'Shape', 'Units', 'Steps', 'Context', 'SIG Stem', 'SIG Options', 'Additional Modifiers']
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        for i in range(1):
            l = reader.next()

        for line in reader:
            error_test = 'FALSE'
            row = {}
            strand_dict = {}
            strand_list = []
            InternalID = int(line['Internal ID'])
            Strand = line['Strand'].split("; ")
            for strand in Strand:
                strand_sub_dict = {}
                content_focus_df = tagging_df['Content_Focus']
                StrandGUID, error_test = get_GUID_lite(strand, content_focus_df, error_test)
                strand_sub_dict = {'Strand': strand, 'StrandGUID' : StrandGUID}
                strand_list.append(strand_sub_dict)
                         
            substrand_list = line['Sub-Strand'].split("; ")
            strand_list, error_test = get_SubstrandGUID(strand_list, substrand_list, content_focus_df, error_test)
                                 
            Differentiator_Keys = ['Shape', 'Units', 'Steps']            
            Differentiator_List = []
            for Differentiator in Differentiator_Keys:
                Differentiator_values = line[Differentiator].split("; ")
                Differentiator_sub_list = []
                Differentiator_df = tagging_df[Differentiator]
                DifferentiatorGUID_Name = Differentiator + "GUID"
                for value in Differentiator_values:
                    Differentiator_sub_dict = {}
                    DifferentiatorGUID, error_test = get_GUID_lite(value, Differentiator_df, error_test)
                    Differentiator_sub_dict = {Differentiator: value, DifferentiatorGUID_Name : DifferentiatorGUID}
                    Differentiator_sub_list.append(Differentiator_sub_dict)
                Differentiator_List.append(Differentiator_sub_list)
            #Process Differentiators
            
            
            row = {"Internal_ID": InternalID, "Organizers" : strand_list, "Differentiators": Differentiator_List}
            if error_test == 'TRUE':
                bad_data.append(row)
            else: data.append(row)
            
            
    return data, bad_data

In [13]:
data, bad_data = process_file(filename, fields, tagging_dict)
pprint.pprint(bad_data)

[{'Differentiators': [[{'Shape': 'Angles',
                        'ShapeGUID': '1E2EB858-97C8-11E6-8048-1FDE29C466BA'},
                       {'Shape': 'Circle',
                        'ShapeGUID': '1E31B990-97C8-11E6-8048-1FDE29C466BA'},
                       {'Shape': 'Line',
                        'ShapeGUID': '1E22E424-97C8-11E6-8048-1FDE29C466BA'},
                       {'Shape': 'Point',
                        'ShapeGUID': '1E24B42A-97C8-11E6-8048-1FDE29C466BA'},
                       {'Shape': 'Right triangle',
                        'ShapeGUID': '1E3C7E3E-97C8-11E6-8048-1FDE29C466BA'},
                       {'Shape': 'Arc',
                        'ShapeGUID': '1E3B9636-97C8-11E6-8048-1FDE29C466BA'}],
                      [{'Units': 'N/A', 'UnitsGUID': 'N/A'}],
                      [{'Steps': 'two Step',
                        'StepsGUID': 'F024BB90-97E2-11E6-9F49-DB8029C466BA'}]],
  'Internal_ID': 333179,
  'Organizers': [{'Strand': 'Geometry and Measurement',
   

In [14]:
len(bad_data)

12

In [15]:
pprint.pprint(data)

[{'Differentiators': [[{'Shape': 'Angles',
                        'ShapeGUID': '1E2EB858-97C8-11E6-8048-1FDE29C466BA'},
                       {'Shape': 'Composite figure',
                        'ShapeGUID': '1E30CF44-97C8-11E6-8048-1FDE29C466BA'},
                       {'Shape': 'Right triangle',
                        'ShapeGUID': '1E3C7E3E-97C8-11E6-8048-1FDE29C466BA'}],
                      [{'Units': 'N/A', 'UnitsGUID': 'N/A'}],
                      [{'Steps': 'N/A', 'StepsGUID': 'N/A'}]],
  'Internal_ID': 331526,
  'Organizers': [{'Strand': 'Geometry and Measurement',
                  'StrandGUID': 'D66D958E-8BFB-11E6-921B-89CC29C466BA',
                  'Substrand': [{'Substrand': 'Shapes',
                                 'SubstrandGUID': '15D1079A-8C17-11E6-81DC-467029C466BA'}]}]}]


In [4]:
def get_SubstrandGUID(strand_list, substrand_list, df, error_test):
    GUID = None
    substrand_sublist = []
    if len(df[(df['Title']==lowercase(substrand_list[0]))]) == 0:  ####THIS JUST TESTS THE FIRST SUBSTRAND FOR EXISTENCE
        substrand_sublist.append({'Substrand': substrand_list[0], 'SubstrandGUID': 'Error: Substrand not found in master list'})
        error_test = 'TRUE'
        strand_list[0]['Substrand'] = substrand_sublist
    elif len(substrand_list) < len(strand_list):
        substrand_sublist.append({'Substrand': substrand_list[0], 'SubstrandGUID': "Error: more strands than substrands"})
        error_test = 'TRUE'
        strand_list[0]['Substrand'] = substrand_sublist
    elif len(substrand_list) == 1 and len(strand_list) == 1:
        if len(df[(df['Strand']==lowercase(strand_list[0]['Strand'])) & (df['Title']==lowercase(substrand_list[0]))]) == 1:
            SubstrandGUID = df.ix[(df['Strand']==lowercase(strand_list[0]['Strand'])) & (df['Title']==lowercase(substrand_list[0])), 'Unique ID'].item()
            substrand_sublist.append({'Substrand': substrand_list[0], 'SubstrandGUID': SubstrandGUID})
            strand_list[0]['Substrand'] = substrand_sublist
        else: 
            strand_list[0]['Substrand']= {'Substrand': substrand_list[0], 'SubstrandGUID': 'Error: No Strand/Substrand match found in master list'}
            error_test = 'TRUE'
            strand_list[0]['Substrand'] = substrand_sublist
    elif len(substrand_list) > 1 and len(strand_list) == 1: 
        for substrand in substrand_list:
            
            if len(df[(df['Strand']==lowercase(strand_list[0]['Strand'])) & (df['Title']==lowercase(substrand))]) == 1:
                SubstrandGUID = df.ix[(df['Strand']==lowercase(strand_list[0]['Strand'])) & (df['Title']==lowercase(substrand)), 'Unique ID'].item()
                substrand_sublist.append({'Substrand': substrand, 'SubstrandGUID': SubstrandGUID})
                strand_list[0]['Substrand'] = substrand_sublist
            else: 
                substrand_sublist.append({'Substrand': substrand, 'SubstrandGUID': 'Error: No Strand/Substrand match found in master list'})
                error_test = 'TRUE'
                strand_list[0]['Substrand'] = substrand_sublist
    elif len(substrand_list) == 2 and len(strand_list) == 2:
        
    
    
    
    
    else: 
        substrand_sublist.append({'Substrand': substrand_list[0], 'SubstrandGUID': "Error: Multiple substrands, cannot parse yet"})
        error_test = 'TRUE'
        strand_list[0]['Substrand'] = substrand_sublist
    return strand_list, error_test


In [12]:
#Case strand len = 1, substrand = 1
strand_list = [{"Strand": 'Geometry and Measurement', "StrandGUID": "testGUID1"}]
error_test = 'FALSE'
substrand = ["Perimeter, Area, Volume", "Measurement"]
data = get_SubstrandGUID(strand_list, substrand, tagging_dict['Content_Focus'], error_test)
pprint.pprint(data)

([{'Strand': 'Geometry and Measurement',
   'StrandGUID': 'testGUID1',
   'Substrand': [{'Substrand': 'Perimeter, Area, Volume',
                  'SubstrandGUID': '15A8ED1E-8C17-11E6-81DC-467029C466BA'},
                 {'Substrand': 'Measurement',
                  'SubstrandGUID': '1586CBB2-8C17-11E6-81DC-467029C466BA'}]}],
 'FALSE')


In [46]:
strand_list = [{"Strand": "Algebra"}, {"Strand": "Geometry and Measurement"}]
substrand_list = ["Time and Money", "Equations"]
strand_names = []
for strand in strand_list:
    strand_names.append(strand['Strand'])
match_status = pd.DataFrame(False,index=strand_names,columns=substrand_list)
df = content_focus_df

for strand in strand_names:
    for substrand in substrand_list:
        if len(df[(df['Strand']==lowercase(strand)) & (df['Title']==lowercase(substrand))]) == 1:
            match_status.ix[strand,substrand] = True
            
print match_status
test = match_status.sum().sum()

if match_status.sum().sum() == 2:
    #get the strand,substrand pairs where match_status is true  
    true_pairs = list(match_status[match_status == True].stack().index)
    #Cycle through pairs and grab GUID
    for pair in true_pairs:
        #strand = pair[0], substrand = pair[1]
        
    #Output the substrand underneath the appropriate strand.
    
    
    
#elif match_status.sum().sum() == 3:
    #figure out what matches
    
    #Output the substrand underneath the appropriate strand.
    


                         Time and Money Equations
Algebra                           False      True
Geometry and Measurement           True     False
[('Algebra', 'Equations'), ('Geometry and Measurement', 'Time and Money')]


In [5]:
def lowercase(text):
    return unicode(text, "latin2").lower()

In [6]:
#load GUID file

In [7]:
#problem to solve for looking up GUID: when strand/substrand gives context to component, 
#if semi-colon separated, then will have to allow for non-matches

In [8]:
def Load_GUIDs(filename):
    data = []
    fields = {}
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        for line in reader:
            data.append(line)
    #strip capitalization for matching purposes
    data = pd.DataFrame(data)
    df_lowercase(data, 'Title')
    if len(data.columns) == 6:
        df_lowercase(data, 'Strand')
        df_lowercase(data, 'Sub-Strand')
    if len(data.columns) == 7:
        df_lowercase(data, 'Strand')
    return data

In [9]:
def df_lowercase(dataframe, column):
    lowercase = lambda x: unicode(x, "latin2").lower()
    dataframe[column] = dataframe[column].apply(lowercase)
    return dataframe

In [10]:
steps_df = Load_GUIDs("Math_Steps.csv")
shapes_df = Load_GUIDs("Math_Shapes.csv")
number_df = Load_GUIDs("Math_Number.csv")
units_df = Load_GUIDs("Math_Units_Modified.csv")
content_focus_df = Load_GUIDs("Math_Content_Foc.csv")
component_df = Load_GUIDs("Math_Components.csv")
tagging_dict = {'Steps': steps_df, 'Shape': shapes_df, 'Number': number_df, 'Units': units_df, 'Content_Focus': content_focus_df, 'Component': component_df}

In [11]:

def get_GUID_lite(tag, df, error_test): 
    #get GUID for steps, number, units, shape
    tag = unicode(tag, "latin2").lower()
    matches = len(df.ix[df['Title'] == tag])
    if matches == 1:
        GUID = df.ix[df['Title'] == tag, 'Unique ID'].item()
    elif matches ==0:
        if tag == r'n/a':
            GUID = r'N/A'
        else: 
            GUID = "Error: no matches found"
            error_test = 'TRUE'
    else: 
        GUID = "Error: resolve multiple values"
        error_test = 'TRUE'
    return GUID, error_test

In [ ]:
number_test = "Square roots"
error_test = "FALSE"
print get_GUID_lite(number_test, number_df, error_test)

In [ ]:

#create version for content focus and component
#Play around with passing multiple conditions into first .ix argument

tag = 'N/A'
if tag == r'N/A':
    print tag


In [ ]:
'''
Data structure

{Internal ID: value
    {Organizers: [
        {Strand: [ {Strand: Strand_value,
                   Strand_GUID: GUID,
                   Substrand: [{Substrand: Substrand_value,
                                Substrand_GUID: GUID,
                                Content_Focus: [{Content_Focus: Content_Focus_value,                       
                                                Content_Focus_GUID: GUID,
                                                Component: [{Component: Component_value,
                                                              Component_GUID: GUID}]}]}]}]
        {Descriptors: {Number: [N1, N2, ...]}
                        {Digits: [D1, D2, ...]}
            {Shape: [Shape1, ...]}
            {Units: [Units1, ...]}
            {Steps: Steps}
            {Context: Context}
            {SIG Stem: [Sig Stem1, ...]}
            {SIG Options: [SIG Options1, ...]}
            {Additional Modifiers: [AM1, ...]}
}

'''